In [1]:
# pip install sparksql-magic

In [1]:
import os
import json
from pyspark.sql import SparkSession
%load_ext sparksql_magic

# Create Spark Session

In [2]:
spark = SparkSession\
    .builder\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
    .getOrCreate()

In [22]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, DateType, TimestampType

# define song schema
song_schema = StructType([
  StructField("artist_id", StringType()),
  StructField("artist_name", StringType()),
  StructField("artist_latitude", DoubleType()),
  StructField("artist_longitude", DoubleType()),
  StructField("artist_location", StringType()),
  StructField("song_id", StringType()),
  StructField("duration", DoubleType()),
  StructField("title", StringType()),
  StructField("year", IntegerType()),
  StructField("num_songs", IntegerType())
])

# read song files
df_song = spark.read.json("data/song_data/A/A/*/*.json", schema=song_schema)
df_song.createOrReplaceTempView("songs_dataset")

# define log schema
log_schema = StructType([
  StructField("artist", StringType()),
  StructField("auth", StringType()),
  StructField("firstName", StringType()),
  StructField("gender", StringType()),
  StructField("itemInSession", IntegerType()),
  StructField("lastName", StringType()),
  StructField("length", DoubleType()),
  StructField("level", StringType()),
  StructField("location", StringType()),
  StructField("method", StringType()),
  StructField("page", StringType()),
  StructField("registration", DoubleType()),
  StructField("sessionId", IntegerType()),
  StructField("song", StringType()),
  StructField("status", IntegerType()),
  StructField("ts", DoubleType()),
  StructField("userAgent", StringType()),
  StructField("userId", StringType())
])

# read log files
df_log = spark.read.json("data/log-data/*.json", schema=log_schema)
df_log.createOrReplaceTempView("logs_dataset")

In [23]:
spark.sql("""
    SELECT DISTINCT
      to_timestamp(l.ts/1000) as start_time,
      YEAR(to_timestamp(l.ts/1000)) as year,
      MONTH(to_timestamp(l.ts/1000)) as month,
      l.userId as user_id,
      l.level as level,
      s.song_id as song_id,
      s.artist_id as artist_id,
      l.sessionId as session_id,
      s.artist_location as artist_location,
      l.userAgent as user_agent
    FROM songs_dataset s
    JOIN logs_dataset l
      ON s.title = l.song 
      AND s.artist_name = l.artist 
      AND s.duration = l.length
    WHERE l.page = 'NextSong'
    AND l.ts IS NOT NULL
    AND l.userId IS NOT NULL
    AND s.song_id IS NOT NULL
  """).toPandas()

start_time  year  month user_id level             song_id  \
0 2018-11-21 21:56:47.796  2018     11      15  paid  SOZCTXZ12AB0182364   

            artist_id  session_id artist_location  \
0  AR5KOSW1187FB35FF4         818       Dubai UAE   

                                          user_agent  
0  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...

# Extracting Cubes

In [3]:
df_users = spark.read.parquet("spark-warehouse/users.parquet")
df_users.printSchema()
df_users.createOrReplaceTempView("users")

df_artists = spark.read.parquet("spark-warehouse/artists.parquet")
df_artists.printSchema()
df_artists.createOrReplaceTempView("artists")

df_songs = spark.read.parquet("spark-warehouse/songs.parquet")
df_songs.printSchema()
df_songs.createOrReplaceTempView("songs")

df_times = spark.read.parquet("spark-warehouse/times.parquet")
df_times.printSchema()
df_times.createOrReplaceTempView("times")

df_songplays = spark.read.parquet("spark-warehouse/songplays.parquet")
df_songplays.printSchema()
df_songplays.createOrReplaceTempView("songplays")

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- duration: double (nullable = true)

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

root
 |-- start_time: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nu

### Time table

In [6]:
%%sparksql -c results
    SELECT * FROM times LIMIT 10

cache dataframe with lazy load
capture dataframe to local variable `results`


### Users table

In [5]:
%%sparksql -c results
    SELECT * FROM users LIMIT 10

cache dataframe with lazy load
capture dataframe to local variable `results`


### Artists table

In [7]:
%%sparksql -c results
    SELECT * FROM artists LIMIT 10

cache dataframe with lazy load
capture dataframe to local variable `results`


### Songs table

In [8]:
%%sparksql -c results
    SELECT * FROM songs LIMIT 10

cache dataframe with lazy load
capture dataframe to local variable `results`


### Songplays table

In [9]:
%%sparksql -c results
    SELECT * FROM songplays LIMIT 10

cache dataframe with lazy load
capture dataframe to local variable `results`


### Users by level

In [23]:
%%sparksql -c results
    SELECT 
        DISTINCT level,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM users
    GROUP BY level

cache dataframe with lazy load
capture dataframe to local variable `results`


### Users by gender

In [24]:
%%sparksql -c results
    SELECT 
        DISTINCT gender,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM users
    GROUP BY gender

cache dataframe with lazy load
capture dataframe to local variable `results`


### Songs by artists

In [25]:
%%sparksql -c results
    SELECT 
        DISTINCT a.artist_name as artist_name,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM songs s
    JOIN artists a ON a.artist_id = s.artist_id
    GROUP BY artist_name

cache dataframe with lazy load
capture dataframe to local variable `results`
only showing top 20 row(s)


### Most executed songs

In [29]:
%%sparksql -c -l 100 results
    SELECT 
        COUNT(sp.*) as executions,
        s.song_id,
        s.title as song_title,
        a.artist_name,
        s.year as song_year
    FROM songplays sp
    JOIN artists a ON a.artist_id = sp.artist_id
    JOIN songs s ON s.song_id = sp.song_id
    GROUP BY s.song_id, song_title, artist_name, song_year
    ORDER BY executions DESC;

cache dataframe with lazy load
capture dataframe to local variable `results`
